In [25]:
"""Minimal jobshop example."""
import collections
from ortools.sat.python import cp_model


def main():
    """Minimal jobshop problem."""
    # Data.
    jobs_data = [  # task = (machine_id, processing_time).
    [(1, 4275245), (2, 3591722), (3, 2551300), (4, 10418268)],  # Job0
    [(1, 2143220), (2, 2661593), (3, 2341746), (4, 7146559)],  # Job1
    [(1, 1736978), (2, 3575178), (3, 2804972), (4, 8117129)],  # Job1
    [(1, 2656078), (2, 612000), (3, 2959381), (4, 6227459)],  # Job1
    [(1, 3175210), (2, 3360109), (3, 2159764), (4, 8695083)],  # Job1
    [(1, 2420788), (2, 3088055), (3, 612000), (4, 6120844)],  # Job1
    [(1, 1939180), (2, 3942818), (3, 2286600), (4, 8168599)],  # Job1
    [(1, 2729606), (2, 2740635), (3, 2540271), (4, 8010513)],  # Job1
    [(1, 2944675), (2, 3521870), (3, 1793962), (4, 8260509)],  # Job1
    [(1, 2257189), (2, 612000), (3, 4091712), (4, 6960901)],  # Job1
    [(1, 612000), (2, 4275532), (3, 2678136), (4, 8522853)],  # Job1
    [(1, 1266220), (2, 3668926), (3, 2538433), (4, 9729230)],  # Job1
    [(1, 1654259), (2, 2343584), (3, 2080721), (4, 6078565)],  # Job1
    [(1, 2338069), (2, 3527385), (3, 3209376), (4, 9970035)],  # Job1
    [(1, 4175220), (2, 3538414), (3, 2547624), (4, 10261259)],  # Job1
    [(1, 2768208), (2, 3703852), (3, 2361966), (4, 8834027)],  # Job1
    [(1, 612000), (2, 3225920), (3, 2367481), (4, 8189869)],  # Job1
    [(1, 2970410), (2, 2788428), (3, 2895044), (4, 9221887)],  # Job1
    [(1, 2054987), (2, 3367461), (3, 612000), (4, 7683314)],  # Job1
    [(1, 2304982), (2, 3135848), (3, 2520051), (4, 8881820)],  # Job1
    [(1, 2545786), (2, 2713062), (3, 2974087), (4, 8475578)]  # Job1
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id + 1].start > all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = assigned_task.job+1
                               # Add spaces to output to align columns.
                sol_line_tasks += '%-10s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Solution:
Optimal Schedule Length: 180008752.0
Machine 0: 
           
Machine 1: 19        17        3         7         13        12        2         10        20        11        14        6         21        4         8         16        9         18        5         15        1         
           [0,2054987]    [2054987,2666987][2666987,4403965][4403965,6343145][6343145,7997404][7997404,9263624][9263624,11406844][11406844,13664033][13664033,15969015][15969015,16581015][16581015,18919084][18919084,21339872][21339872,23885658][23885658,26541736][26541736,29271342][29271342,32039550][32039550,34984225][34984225,37954635][37954635,41129845][41129845,45305065][45305065,49580310]
Machine 2: 19        17        13        2         10        20        14        6         4         21        8         3         9         18        5         15        1         12        16        7         11        
           [2054988,5422449][5422449,8648369][8648369,10991953][11406845,14068438][140684